### LIBRARY IMPORTS

In [1]:
from langchain.document_loaders import UnstructuredURLLoader

In [2]:
loader =UnstructuredURLLoader(
    urls=[
        "https://edition.cnn.com/2024/10/01/media/cnn-launch-digital-subscription-paywall/index.html",
        "https://www.nature.com/articles/d41586-024-03169-9"
    ]
)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
data = loader.load()
len(data)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


2

In [4]:
txt=""
list_txt =[]
for i in range(len(data)):
    txt+='\n'+data[i].page_content
    list_txt.append(txt)

In [5]:
from langchain.text_splitter import CharacterTextSplitter
splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

In [6]:
chunks = splitter.split_text(txt)

In [7]:
len([len(chunk) for chunk in chunks])

20

In [8]:
from chromadb import Client
from sentence_transformers import SentenceTransformer

c:\Users\ADMIN\PROJECTS\NLP-PROJECTS\news-analysis\env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
client = Client()
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\ADMIN\PROJECTS\NLP-PROJECTS\news-analysis\env\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
collection = client.create_collection(name='my_embeddings_collection')
embeddings = model.encode(chunks, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]


In [14]:
len(embeddings)

20

In [15]:
ids = [f'doc_{i}' for i in range(len(chunks))]

# Insert text and embeddings into the collection
for text, embedding, id_ in zip(chunks, embeddings, ids):
    collection.add(
        documents=[text],
        embeddings=[embedding.tolist()],  # Convert numpy array to list
        ids=[id_]                         # Add unique ID for each document
    )

: 

In [44]:
query_text = "ChatGPt"
query_embedding = model.encode(query_text)

# Query the collection
results = collection.query(
    query_embeddings=[query_embedding.tolist()],  # Convert to list
    n_results=1                                   # Number of results to return
)

In [ ]:
results['documents']